In [29]:
!pip install supervision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 2.2 MB/s eta 0:00:00a 0:00:01


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# === Paths ===
model_path = "best.pt"
video_path = "15sec_input_720p.mp4"
output_video_path = "players_only_boxes_with_ids_filtered.mp4"

# === Load YOLO model ===
model = YOLO(model_path)

# === Identify 'player' class ID ===
player_class_id = None
for cls_id, cls_name in model.names.items():
    if cls_name.lower() == "player":
        player_class_id = cls_id
        break

if player_class_id is None:
    raise ValueError("'player' class not found in model.names")

print(f" Found player class ID: {player_class_id}")

# === Video Setup ===
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_video_path,
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps,
                      (frame_width, frame_height))

next_id = 0
players = {}  # player_id: [centroid_x, centroid_y]
max_distance = 50  # pixel threshold for ID matching

def get_centroid(box):
    x1, y1, x2, y2 = map(int, box)
    return ((x1 + x2) // 2, (y1 + y2) // 2)

def match_centroid(new_centroid):
    global next_id
    for pid, prev_centroid in players.items():
        dist = np.linalg.norm(np.array(new_centroid) - np.array(prev_centroid))
        if dist < max_distance:
            players[pid] = new_centroid
            return pid
    players[next_id] = new_centroid
    next_id += 1
    return next_id - 1

def is_referee_color(crop_img):
    hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([35, 255, 255])
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    yellow_ratio = np.sum(mask > 0) / (crop_img.shape[0] * crop_img.shape[1])
    return yellow_ratio > 0.4  # 40% yellow is considered referee

# === Frame Loop ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    for result in results.boxes:
        cls_id = int(result.cls[0])
        if cls_id != player_class_id:
            continue

        box = result.xyxy[0].tolist()
        x1, y1, x2, y2 = map(int, box)
        crop = frame[y1:y2, x1:x2]

        # === Skip if likely referee (yellow-dominant) ===
        if crop.size == 0 or is_referee_color(crop):
            continue

        centroid = get_centroid(box)
        player_id = match_centroid(centroid)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f"ID: {player_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print("✅ Filtered players detected (referee ignored).")
print(f"🎞️ Output saved to: {output_video_path}")


✅ Found player class ID: 2

0: 384x640 1 ball, 16 players, 2 referees, 25.5ms
Speed: 2.5ms preprocess, 25.5ms inference, 17.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 18.2ms
Speed: 1.9ms preprocess, 18.2ms inference, 17.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 15.7ms
Speed: 2.1ms preprocess, 15.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 13.1ms
Speed: 1.3ms preprocess, 13.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 13.5ms
Speed: 1.2ms preprocess, 13.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 13.3ms
Speed: 1.1ms preprocess, 13.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 13.3ms
Speed: 1.3ms preprocess, 13.3ms inference, 1.4ms

In [13]:
!pip install filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=63dfc3fd410d762c192956d66d8e8e7bbab5bdb6f16cbdda08ff134b7f3514e4
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
